In [1]:
import numpy as np
import pandas as pd
import nltk
import re

PREP BEFORE BAYES COMES

In [2]:
df = pd.read_csv('spam.csv', encoding='Latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df = df[['v1','v2']]
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
reg_for_digits = r'\d+'
reg_for_punctuation = r'[^\w\s]'
df['v2'] = df['v2'].replace(reg_for_digits, '',regex=True)
df['v2'] = df['v2'].replace(reg_for_punctuation, '',regex=True)
df['v2'] = df['v2'].str.lower()
df.head()


,v1,v2
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in a wkly comp to win fa cup final...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


In [5]:
reg_for_chars = r'[^a-zA-Z0-9\s]'
no_chars = []
for text in df['v2']:
    sol_chars = re.sub(reg_for_chars, '',text)
    no_chars.append(sol_chars)
df['v2'] = no_chars
df.head()

,v1,v2
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in a wkly comp to win fa cup final...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vardanarakelyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vardanarakelyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vardanarakelyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/vardanarakelyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:

df['v2'] = df['v2'].apply(word_tokenize)

stop_words = set(stopwords.words('english'))
custom_words = {'u', 'ur', 'im', 'lor', 'da', 'ok', 'dat', 'wat', 'got', 'get', 'go', 'like', 'call', 'know', 'come', 'lt', 'gt', 'ill'}
stop_words.update(custom_words)


def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]


df['v2'] = df['v2'].apply(remove_stopwords)

In [8]:
df.head()

,v1,v2
0,ham,"[jurong, point, crazy, available, bugis, n, gr..."
1,ham,"[lar, joking, wif, oni]"
2,spam,"[free, entry, wkly, comp, win, fa, cup, final,..."
3,ham,"[dun, say, early, hor, c, already, say]"
4,ham,"[nah, dont, think, goes, usf, lives, around, t..."


In [9]:
word_list = []
for doc in df['v2']:
    for word in doc:
        word_list.append(word)
vocabulary = set(word_list)
word_index = {word: i for i, word in enumerate(vocabulary)}
print(word_index)

{'eyes': 0, 'problematic': 1, 'fusion': 2, 'knw': 3, 'forgets': 4, 'cine': 5, 'retired': 6, 'alex': 7, 'oxygen': 8, 'sura': 9, 'sane': 10, 'important': 11, 'bloomberg': 12, 'hopeafternoon': 13, 'subtoitles': 14, 'glasgow': 15, 'temp': 16, 'scallies': 17, 'hey': 18, 'showroomscity': 19, 'gudk': 20, 'combine': 21, 'olayiwolas': 22, 'meis': 23, 'starti': 24, 'msgs': 25, 'successfully': 26, 'title': 27, 'dislikes': 28, 'nicenicehow': 29, 'dizzamn': 30, 'transfered': 31, 'harishs': 32, 'help': 33, 'version': 34, 'dad': 35, 'hourish': 36, 'global': 37, 'sth': 38, 'appt': 39, 'ideas': 40, 'dodgey': 41, 'compliments': 42, 'madodu': 43, 'posh': 44, 'thekingshead': 45, 'curtsey': 46, 'reach': 47, 'shldxxxx': 48, 'gossip': 49, 'sathya': 50, 'keypad': 51, 'regarding': 52, 'nora': 53, 'feng': 54, 'abroad': 55, 'smsing': 56, 'worc': 57, 'email': 58, 'trends': 59, 'careless': 60, 'simple': 61, 'weaknesses': 62, 'sarcastic': 63, 'aspects': 64, 'cleared': 65, 'mon': 66, 'reboot': 67, 'tau': 68, 'correc

In [10]:
def bow_vector(tokens):
    vector = np.zeros(len(word_index))
    for token in tokens:
        if token in word_index:
            vector[word_index[token]] += 1
    return vector

df['bow_vector'] = df['v2'].apply(bow_vector)
df.head()

,v1,v2,bow_vector
0,ham,"[jurong, point, crazy, available, bugis, n, gr...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
1,ham,"[lar, joking, wif, oni]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,spam,"[free, entry, wkly, comp, win, fa, cup, final,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,ham,"[dun, say, early, hor, c, already, say]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,ham,"[nah, dont, think, goes, usf, lives, around, t...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


NAIVE BAYES IMPLEMENTATION

In [13]:
p_spam = len(df[df['v1'] == 'spam']) / len(df['v1'])
p_ham = len(df[df['v1'] == 'ham']) / len(df['v1'])

In [14]:
print(p_spam, p_ham)

0.13406317300789664 0.8659368269921034


In [15]:
spam_messages = df[df['v1'] == 'spam']['v2']
ham_messages = df[df['v1'] == 'ham']['v2']

In [18]:
print(spam_messages)

2       [free, entry, wkly, comp, win, fa, cup, final,...
5       [freemsg, hey, darling, weeks, word, back, id,...
8       [winner, valued, network, customer, selected, ...
9       [mobile, months, r, entitled, update, latest, ...
11      [six, chances, win, cash, pounds, txt, csh, se...
                              ...                        
5537    [want, explicit, sex, secs, ring, costs, pmin,...
5540    [asked, mobile, chatlines, inclu, free, mins, ...
5547    [contract, mobile, mnths, latest, motorola, no...
5566    [reminder, pounds, free, credit, details, grea...
5567    [nd, time, tried, contact, pound, prize, claim...
Name: v2, Length: 747, dtype: object


In [19]:
spam_words = [word for msg in spam_messages for word in msg]
print(spam_words)

['free', 'entry', 'wkly', 'comp', 'win', 'fa', 'cup', 'final', 'tkts', 'st', 'may', 'text', 'fa', 'receive', 'entry', 'questionstd', 'txt', 'ratetcs', 'apply', 'overs', 'freemsg', 'hey', 'darling', 'weeks', 'word', 'back', 'id', 'fun', 'still', 'tb', 'xxx', 'std', 'chgs', 'send', 'rcv', 'winner', 'valued', 'network', 'customer', 'selected', 'receivea', 'prize', 'reward', 'claim', 'claim', 'code', 'kl', 'valid', 'hours', 'mobile', 'months', 'r', 'entitled', 'update', 'latest', 'colour', 'mobiles', 'camera', 'free', 'mobile', 'update', 'co', 'free', 'six', 'chances', 'win', 'cash', 'pounds', 'txt', 'csh', 'send', 'cost', 'pday', 'days', 'tsandcs', 'apply', 'reply', 'hl', 'info', 'urgent', 'week', 'free', 'membership', 'prize', 'jackpot', 'txt', 'word', 'claim', 'tc', 'wwwdbuknet', 'lccltd', 'pobox', 'ldnwarw', 'xxxmobilemovieclub', 'use', 'credit', 'click', 'wap', 'link', 'next', 'txt', 'message', 'click', 'httpwap', 'xxxmobilemovieclubcomnqjkgighjjgcbl', 'england', 'v', 'macedonia', 'do

In [22]:
ham_words = [word for msg in ham_messages for word in msg]

In [26]:
def counter(to_find):
    word_count = {}
    for word in to_find:
        if word in word_count:
            word_count[word] += 1
        else:
            word_count[word] = 1
    return word_count

In [27]:
spam_counter = counter(spam_words)
ham_counter = counter(ham_words)

In [28]:
print(spam_counter)

{'free': 219, 'entry': 26, 'wkly': 14, 'comp': 10, 'win': 61, 'fa': 4, 'cup': 5, 'final': 16, 'tkts': 4, 'st': 30, 'may': 7, 'text': 121, 'receive': 33, 'questionstd': 2, 'txt': 151, 'ratetcs': 2, 'apply': 30, 'overs': 2, 'freemsg': 12, 'hey': 5, 'darling': 2, 'weeks': 13, 'word': 22, 'back': 23, 'id': 3, 'fun': 9, 'still': 7, 'tb': 1, 'xxx': 12, 'std': 9, 'chgs': 1, 'send': 67, 'rcv': 4, 'winner': 16, 'valued': 11, 'network': 25, 'customer': 49, 'selected': 25, 'receivea': 2, 'prize': 92, 'reward': 10, 'claim': 113, 'code': 29, 'kl': 2, 'valid': 23, 'hours': 6, 'mobile': 124, 'months': 6, 'r': 23, 'entitled': 8, 'update': 19, 'latest': 36, 'colour': 17, 'mobiles': 12, 'camera': 33, 'co': 5, 'six': 3, 'chances': 2, 'cash': 62, 'pounds': 17, 'csh': 2, 'cost': 23, 'pday': 2, 'days': 16, 'tsandcs': 2, 'reply': 101, 'hl': 8, 'info': 12, 'urgent': 63, 'week': 50, 'membership': 2, 'jackpot': 2, 'tc': 21, 'wwwdbuknet': 2, 'lccltd': 2, 'pobox': 30, 'ldnwarw': 2, 'xxxmobilemovieclub': 1, 'use':

In [29]:
vocab_size = len(vocabulary)

In [30]:
spam_length = len(spam_words)
ham_length = len(ham_words)

In [31]:
spam_likelihood = {}
ham_likelihood = {}

In [33]:
for word in vocabulary:
    spam_likelihood[word] = (spam_counter.get(word, 0) + 1) / (spam_length + vocab_size)

    ham_likelihood[word] = (ham_counter.get(word, 0) + 1) / (ham_length + vocab_size)


In [36]:
print('likelihood of free being spam:', spam_likelihood['free'])
print('likelihood of free being ham:', ham_likelihood['free'])

likelihood of free being spam: 0.012085255987695012
likelihood of free being ham: 0.0013983732258139698


In [38]:

def classify(message):
    message = message.lower()
    tokens = nltk.word_tokenize(message)
    spam_score = np.log(p_spam)
    ham_score = np.log(p_ham)

    for word in tokens:
        if word not in spam_likelihood and word not in ham_likelihood:
            pass
        else:
            spam_score += np.log(spam_likelihood[word])
            ham_score += np.log(ham_likelihood[word])
    if spam_score > ham_score:
        return 'spam'
    if spam_score < ham_score:
        return 'ham'



message_1 = "Congratulations! You've won a free cruise to the Bahamas. Text WIN to 88888 to claim your prize now!"

classify_1 = classify(message_1)
print(classify_1)

message_2 = 'Hey, are you still coming to the meeting at 3pm? Let me know.'

classify_2 = classify(message_2)
print(classify_2)

spam
ham
